In [1]:
import pandas as pd
import numpy as np
import os
import re
import xgboost as xgb
import matplotlib.pyplot as plt

C:\Users\gbao\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
path = r"C:\Users\gbao\Google 云端硬盘\BUDT758T\Project\Data"
data_dict = pd.read_excel(os.path.join(path,"data_dictionary_2020.xlsx"),index_col=0)
X_train = pd.read_csv(os.path.join(path,"airbnb_train_x.csv"),index_col=0)
y_train = pd.read_csv(os.path.join(path,"airbnb_train_y.csv"),index_col=0)
X_test = pd.read_csv(os.path.join(path,"airbnb_test_x.csv"),index_col=0)

#whole_data = X_whole.merge(y_train,how="left",left_index=True,right_index=True)
test_index = X_test.index
print(X_train.shape)
print(X_test.shape)
HPI = pd.read_excel(os.path.join(path,"HPI_AT_BDL_county.xlsx"), skiprows=6)

X_all = pd.concat([X_train,X_test],keys=["train","test"])
HPI = HPI.groupby(["FIPS code"]).max()

C:\Users\gbao\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4,6,7,8,10,11,28,35,43,45,69) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(100000, 69)
(12208, 69)


In [3]:
ohe_features = ["bed_type","cancellation_policy","experiences_offered",
                "host_has_profile_pic","host_identity_verified","host_is_superhost",
               "host_response_time","host_neighbourhood","host_verifications",
               "instant_bookable","is_business_travel_ready","is_location_exact",
               "jurisdiction_names","license","market","neighbourhood",
               "property_type","require_guest_phone_verification","require_guest_profile_picture",
               "requires_license","room_type","smart_location"]

features = ["amenities","bed_type","cancellation_policy",
                "host_has_profile_pic","host_identity_verified","host_is_superhost",
               "host_response_time","host_verifications",
               "instant_bookable","is_business_travel_ready","is_location_exact",
               "property_type","require_guest_phone_verification","require_guest_profile_picture",
               "requires_license","room_type"]


In [4]:
#data processing function
def data_process(data, features):
    data["amenities"] = data.amenities.str.split(",").apply(len)
    quantile_25 = data.amenities.quantile(0.25)
    quantile_75 = data.amenities.quantile(0.75)
    data["amenities"] = data.amenities.apply(lambda x:"Low" if x<quantile_25 else ("Hihg" if x>quantile_25 else "Middel"))
    data["host_verifications"] = data.host_verifications.astype("str")
    data["host_verifications"] = data.host_verifications.str.split(",")
    data["host_verifications"] = data["host_verifications"].apply(len)
    quantile_25 = data.host_verifications.quantile(0.25)
    quantile_75 = data.host_verifications.quantile(0.75)
    data["host_verifications"] = data.host_verifications.apply(lambda x:"Low" if x<quantile_25 else ("Hihg" if x>quantile_25 else "Middel"))
    
    valid_element = ['Real Bed','Futon','Couch','Pull-out Sofa','Airbed']
    data.bed_type = data.bed_type.apply(lambda x: np.nan if x not in valid_element else x)
    valid_element = ['strict','moderate','flexible','super_strict_30','super_strict_60']
    data.cancellation_policy = data.cancellation_policy.apply(lambda x: np.nan if x not in valid_element else x)
    valid_element = ['t','f']
    data.host_has_profile_pic = data.host_has_profile_pic.apply(lambda x: np.nan if x not in valid_element else x)
    data.host_identity_verified = data.host_identity_verified.apply(lambda x: np.nan if x not in valid_element else x)
    data.host_is_superhost = data.host_is_superhost.apply(lambda x: np.nan if x not in valid_element else x)
    data.instant_bookable = data.instant_bookable.apply(lambda x: np.nan if x not in valid_element else x)
    data.is_business_travel_ready = data.is_business_travel_ready.apply(lambda x: np.nan if x not in valid_element else x)
    data.is_location_exact = data.is_location_exact.apply(lambda x: np.nan if x not in valid_element else x)
    data.require_guest_phone_verification = data.require_guest_phone_verification.apply(lambda x: np.nan if x not in valid_element else x)
    valid_element = ['within a few hours','within an hour','within a day','a few days or more']
    data.host_response_time = data.host_response_time.apply(lambda x: np.nan if x not in valid_element else x)
    
    pattern = "(\d{5})"
    data.zipcode = data.zipcode.astype("str", copy=False)
    data.zipcode = data.zipcode.str.extract(pattern)
    data.zipcode = data.zipcode.astype("str", copy=False)
    data.zipcode = data.zipcode.apply(lambda x: x[:4])
    
    numerical_column = data_dict[data_dict["variable type"] == "numerical"].index.values
    
    pattern = "(\d+\.*\d*)"
    for i in numerical_column:
        temp = data[i].astype("str")
        temp = temp.str.extract(pattern)
        data[i] = temp.astype("float")
        
    temp = features+list(numerical_column)
    data = data[temp]

    #data.dropna(thresh=15, inplace=True)
    for i in features:
        temp = pd.get_dummies(data[i], prefix = i)
        data.drop([i],axis=1,inplace=True)
        data = pd.concat((data, temp), axis=1)
    return(data)


X_all = data_process(X_all, features)
X_train = X_all.loc["train"]
X_train.dropna(thresh=15, inplace=True)
X_test = X_all.loc["test",:]

C:\Users\gbao\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\gbao\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
def xgb_data_process(X_train_processed, y_train):
    temp = X_train_processed.merge(y_train,how='left',left_index=True,right_index=True)
    temp.dropna(subset=["high_booking_rate"],inplace=True)
    columns=y_train.columns
    y_train_xgb = temp.high_booking_rate
    X_train_xgb = temp.drop(columns=columns)
    dtrain = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
    return(dtrain)

dtrain = xgb_data_process(X_train, y_train)

param = {"max_depth": 6, "eta":1,"objective":"binary:logistic"}
num_round = 1000
#bst = xgb.cv(param, dtrain, num_boost_round=30, nfold=5, seed=52)

In [6]:
record_errors = np.array([])
grid_depth = np.arange(1,100,5)
for depth in grid_depth:
    param = {"max_depth": depth, "eta":1,"objective":"binary:logistic"}
    bst = xgb.cv(param, dtrain, num_boost_round=30, nfold=5, seed=42, early_stopping_rounds=20) 
    record_errors = np.append(record_errors, bst.iloc[-1,2])
best_index = np.where(record_errors == np.min(record_errors))

In [7]:
param = {"max_depth": int(grid_depth[best_index]), "eta":1,"objective":"binary:logistic"}
watch_list = [(dtrain,"train")]
bst = xgb.train(param, dtrain, num_boost_round=30,early_stopping_rounds=20,evals=watch_list)
dtest = xgb.DMatrix(X_test)
ypred = bst.predict(dtest)
result = pd.Series(map(lambda x: 1 if x>0.5 else 0, ypred),name="high_booking_rate")
len(result)
result.to_excel("high_booking_rate.xlsx") 

[0]	train-error:0.21258
Will train until train-error hasn't improved in 20 rounds.
[1]	train-error:0.19149
[2]	train-error:0.18612
[3]	train-error:0.18105
[4]	train-error:0.17646
[5]	train-error:0.17289
[6]	train-error:0.17101
[7]	train-error:0.16757
[8]	train-error:0.16404
[9]	train-error:0.16342
[10]	train-error:0.16077
[11]	train-error:0.15976
[12]	train-error:0.15737
[13]	train-error:0.15649
[14]	train-error:0.15520
[15]	train-error:0.15403
[16]	train-error:0.15233
[17]	train-error:0.15048
[18]	train-error:0.14915
[19]	train-error:0.14704
[20]	train-error:0.14615
[21]	train-error:0.14540
[22]	train-error:0.14511
[23]	train-error:0.14361
[24]	train-error:0.14249
[25]	train-error:0.14183
[26]	train-error:0.13997
[27]	train-error:0.13861
[28]	train-error:0.13775
[29]	train-error:0.13626
